In [1]:
import tkinter as tk
import math
import threading
import time
import requests
import os

ip = "192.168.1.103"

path = "/Users/jashan-ck/Downloads/GIT/Summer_Project/jupyter notebook/Voice Based"

working_path = os.getcwd()
dir_back_from_cwd = 1

w_modes = ['file', 'coding', 'run', 'selection', 'scroll', 'delete']

working_mode = 'scroll'

parent_dirs = []
for i,w in enumerate(working_path):
    if '/' == w :
        parent_dirs.append(i)
parent_dirs
working_path = working_path[:parent_dirs[-1]]

In [2]:
def server_data_fetch():
    global home_thread,pill2kill
    s_c_link = "http://" + ip + ":8000/check_status/"
    g_c_d_link = "http://" + ip + ":8000/get_command_data/"

    loop = True
    while loop:
        status_check = int(requests.get(s_c_link).text)
        if status_check : 
            command = requests.get(g_c_d_link).text.strip()
            command = command[1:-1]
            print(command)
            if command == 'exit loop' :
                loop = False
            elif command == 'wakeup':
                open_file_mode_canvas()
                global thread_list
                pill2kill.set()
                for i in range(len(thread_list)):
                    thread_list[i].join()
            elif command == 'go home':
                open_home_canvas()
            else :
                command_analyser(command)

threading.Thread(target=server_data_fetch).start()

In [3]:
def create_circle_wire_frame_data(x, y, d, pt):
    l = []
    l.append((x+d//2, y+0))
    l.append((x+0, y+d//2))
    l.append((x+d//2, y+d))
    l.append((x+d, y+d//2))
    x45 = d//2 * math.sin(math.radians(45))
    y45 = d//2 * math.cos(math.radians(45))
    l.insert(1,(x+d//2 - x45, y+d//2 - y45))
    l.insert(3,(x+d//2 - x45, y+d//2 + y45))
    l.insert(5,(x+d//2 + x45, y+d//2 + y45))
    l.insert(7,(x+d//2 + x45, y+d//2 - y45))
    if pt > 8:
        l_ang = [22.5,67.5]
        l_pos = [[1,5,7,14],[3,5,11,13]]
        for i in range(2):
            x45 = d//2 * math.sin(math.radians(l_ang[i]))
            y45 = d//2 * math.cos(math.radians(l_ang[i]))
            l.insert(l_pos[i][0], (x+d//2 - x45, y+d//2 - y45))
            l.insert(l_pos[i][1], (x+d//2 - x45, y+d//2 + y45))
            l.insert(l_pos[i][2], (x+d//2 + x45, y+d//2 + y45))
            l.insert(l_pos[i][3], (x+d//2 + x45, y+d//2 - y45))
    return l

def sub_wire_frame_dig(c_in, c_out):
    sub = []
    for i in range(len(c_in)):
        sub.append(c_in[i])
        sub.append(c_out[i*2 + 1])
    return sub

def sub_wire_frame_larg_dig(c_in, c_out):
    sub = []
    for i in range(0,len(c_in),2):
        sub.append(c_in[i])
        sub.append(c_out[(i+1)*2])
    return sub

def sub_wire_frame_str(c_in, c_out):
    sub = []
    for i in range(len(c_in)):
        sub.append(c_in[i])
        sub.append(c_out[i*2])
    return sub

def create_wire_frame(parent_canv, pt_list):
    pt1 = pt_list[0]
    for i in range(1,len(pt_list)):
        pt2 = pt_list[i]
        parent_canv.create_line(pt1[0], pt1[1], pt2[0], pt2[1], fill="#888888", width=2)
        pt1 = pt2
    parent_canv.create_line(pt_list[0][0], pt_list[0][1], pt_list[-1][0], pt_list[-1][1], fill="#888888", width=2)
    
def create_wire_frame_breaked(parent_canv, pt_list):
    for i in range(0,len(pt_list)-1,2):
        pt1 = pt_list[i]
        pt2 = pt_list[i+1]
        parent_canv.create_line(pt1[0], pt1[1], pt2[0], pt2[1], fill="#888888", width=2)
        
def thread_anim(stop_event, parent_canv, pt_list, s):
    color_l = ["#000000", "#00FFFF"]
    c = True
    while not stop_event.wait(1):
        for i in range(len(pt_list)-1):
            pt1 = pt_list[i]
            pt2 = pt_list[i+1]
            parent_canv.create_line(pt1[0], pt1[1], pt2[0], pt2[1], fill=color_l[int(c)], width=2)
            time.sleep(s)
        parent_canv.create_line(pt_list[0][0], pt_list[0][1], pt_list[-1][0], pt_list[-1][1], fill=color_l[int(not c)], width=2)
        c = not c
        print('.')
        
def thread_anim_str(stop_event, parent_canv, pt_list, s):
    color_l = ["#000000", "#00FFFF"]
    c = True
    while not stop_event.wait(1):
        for i in range(0,len(pt_list)-1,2):
            pt1 = pt_list[i]
            pt2 = pt_list[i+1]
            parent_canv.create_line(pt1[0], pt1[1], pt2[0], pt2[1], fill=color_l[int(c)], width=2)
            time.sleep(s)
        parent_canv.create_line(pt_list[0][0], pt_list[0][1], pt_list[-1][0], pt_list[-1][1], fill=color_l[int(not c)], width=2)
        c = not c
        print("#")

In [4]:
# visibility function for the canvas

def open_file_mode_canvas():
    global startup_canvas,canvas_file,w,h
    startup_canvas.config(width=0, height=0, bg="#ffffff")
    canvas_file.config(width=w, height=h, bg="#ffffff")
    
def open_home_canvas():
    global startup_canvas,canvas_file,w,h
    canvas_file.config(width=0, height=0, bg="#000000")
    startup_canvas.config(width=w, height=h, bg="#000000")
    start_thread_animation_work()
    
def start_thread_animation_work():
    global thread_list,pill2kill,startup_canvas,circle_inner,circle_outer
    pill2kill = threading.Event()
    th_dig = threading.Thread(target=thread_anim, args=(pill2kill, startup_canvas,sub_wire_frame_dig(circle_inner, circle_outer), 0.01))
    th_cir_in = threading.Thread(target=thread_anim, args=(pill2kill, startup_canvas, circle_inner, 0.05))
    th_cir_out = threading.Thread(target=thread_anim, args=(pill2kill, startup_canvas,circle_outer, 0.02))
    th_dig_l = threading.Thread(target=thread_anim, args=(pill2kill, startup_canvas,sub_wire_frame_larg_dig(circle_inner, circle_outer), 0.03))
    th_str = threading.Thread(target=thread_anim_str, args=(pill2kill, startup_canvas, sub_wire_frame_str(circle_inner, circle_outer), 0.02))
    thread_list = []
    thread_list.append(th_dig)
    thread_list.append(th_cir_in)
    thread_list.append(th_cir_out)
    thread_list.append(th_dig_l)
    thread_list.append(th_str)
    
    for i in range(len(thread_list)):
        thread_list[i].start()
        

In [ ]:
# canvas FILES 
# 

def list_files_in_current_dir() :
    return os.listdir(working_path)

def write_multiline_txt(txt, limit):
    length = len(txt)
    s = ""
    for i,w in enumerate(txt):
        if i % limit == 0:
            s += '\n'
        s += w
    return s

def go_back():
    global working_path,dir_back_from_cwd,parent_canvas
    dir_back_from_cwd += 1
    working_path = working_path[:parent_dirs[-dir_back_from_cwd]]
#     parent_canvas.itemconfig('working_path', text=working_path)
    
def open_folder(folder):
    global working_path,dir_back_from_cwd
    working_path += '/' + folder
    dir_back_from_cwd -= 1
    
    
def command_analyser(command):
    print(command)
    if command == 'go back':
        print("in file mode go back")
        go_back()
        refresh_files_view()
    print(command[:4])
    if command[:4] == 'open':
        folder = command[5:]
        d = list_files_in_current_dir()
#         for i,w in enumerate(d):
#             if '.' in w:
#                 del d[i]
        print(folder)
        if folder not in d:
            print("Error No such Folder")
        else :
            print(folder,'.')
            open_folder(folder)
            refresh_files_view()
        
def refresh_files_view():
    global canvas_file,parent_canvas
    canvas_file.destroy()
    canvas_file = tk.Canvas(parent_canvas, width=700 , height=700 , bg = "#ffffff")
    parent_canvas.create_window(5, 5, anchor= tk.NW, window=canvas_file)
    dir_list = list_files_in_current_dir()
    
    for i,val in enumerate(dir_list):
        y = 0 + (i // 7) * 100
        x = 0 + i * 100 - (y * 7)
        canvas_file_obj = tk.Canvas(canvas_file, width=100, height=100)
        val = str(val)
        if '.' in val :
            canvas_file_obj.create_image(25, 25, image=img_file, anchor = tk.NW, tags="file")
        else :
            canvas_file_obj.create_image(25, 25, image=img_folder, anchor = tk.NW, tags="folder")

        t = canvas_file_obj.create_text(50, 80, text=write_multiline_txt(val,15), fill="#666666", font="Time 11")
        canvas_file.create_window(x, y, anchor=tk.NW, window=canvas_file_obj)
     

In [ ]:
window = tk.Tk()

w = 1190
h = 770

w_file = 700
h_file = 700

def create_rounded_rect(canvas, x1, y1, x2, y2, **kwargs ) :
    h = y2 - y1
    canvas.create_oval( x1, y1, x1+h, y1+h, outline="", **kwargs )
    canvas.create_line( x1+h/2, y1 + h/2, x2-h/2, y1 + h/2, width=h, **kwargs)
    canvas.create_oval( x2 - h, y2 - h, x2, y2, outline="", **kwargs )

def call_command(event) :
    action = event.widget.get().lower().strip()
    command_analyser(action)

parent_canvas = tk.Canvas(window, width=w, height=h)

startup_canvas = tk.Canvas(parent_canvas, width=w, height=h, bg="#000000")
parent_canvas.create_window(0, 0, window=startup_canvas, anchor=tk.NW)

circle_inner = create_circle_wire_frame_data(400, 200, 400, 8)
circle_outer = create_circle_wire_frame_data(300, 100, 600, 16)

create_wire_frame(startup_canvas, circle_inner)
create_wire_frame(startup_canvas, circle_outer)

create_wire_frame(startup_canvas, sub_wire_frame_dig(circle_inner, circle_outer))
create_wire_frame(startup_canvas, sub_wire_frame_larg_dig(circle_inner, circle_outer))
create_wire_frame_breaked(startup_canvas, sub_wire_frame_str(circle_inner, circle_outer))



start_thread_animation_work()

    
# canvas for file view
#
# show files
canvas_file = tk.Canvas(parent_canvas, width=700 , height=700 , bg = "#dddddd")
parent_canvas.create_window(5, 5, anchor= tk.NW, window=canvas_file)

# creating a Design for file icon with name
img_folder = tk.PhotoImage(file=path + '/assets/folder.png', master=parent_canvas)
img_file = tk.PhotoImage(file=path + '/assets/file.png', master=parent_canvas)

dir_list = list_files_in_current_dir()

for i,val in enumerate(dir_list):
    y = 0 + (i // 7) * 100
    x = 0 + i * 100 - (y * 7)
    canvas_file_obj = tk.Canvas(canvas_file, width=100, height=100)
    val = str(val)
    if '.' in val :
        canvas_file_obj.create_image(25, 25, image=img_file, anchor = tk.NW, tags="file")
    else :
        canvas_file_obj.create_image(25, 25, image=img_folder, anchor = tk.NW, tags="folder")
            
    t = canvas_file_obj.create_text(50, 80, text=write_multiline_txt(val,15), fill="#666666", font="Time 11")
    canvas_file.create_window(x, y, anchor=tk.NW, window=canvas_file_obj)
    
# parent_canvas.create_text(350, 730, text=working_path, fill="#222222", font='Time 15', tags="working_path")


open_home_canvas()

parent_canvas.pack()
tk.mainloop()

.
.
#
#
.
.
.
.
.
.
.
#
.
#
.
.
.
.
.
.
#
.
.
#
.
.
.
.
.
.
#
.
.
#
.
.
.
.
.
.
#
.
#
.
.
.
.
.
.
.
#
#.

.
..

.
.
.
.
#
#
.
.
..

.
#
.
#.
.

.
.
.
.
#
.
#
.
.
..

.
.
.
#
#
.
.
.
.
.
.
.
.
#
#
.
.
.
.
.
.
.
.
#
#
.
.
.
.
.
.
.
#
.
#
.
.
.
.
#
.#

.
.
.
.
.
.
.
#
#
.
.
..

.
.
.
.
#
#
.
.
.
.
..

.
.
#
#
.
.
.
.
#
.
.
.
.
#
.
.
.
.
#
.
..
#

.
.
.
.
.
#
#
.
..

.
.
.
.
#
.
#
.
.
.
.
.
.
#
.
#
.
.
.
.
.
.
.
#
#
.
..

.
.
#
.
.
.
#
.
..

.
.
#
.
.
.
#
.
.
.
.
.
#
.
.
#
.
.
.
.
#
.
.
.
.
#
.
.
.
#
.
.
..

#
.
.
.
.
#
.
.
#
.
.
.
.
.
.
#
.
.
#
.
..

.
.
#
.
.
#
.
.
.
.
#
.
.
.
#
.
.
.
.
.
#
.
#
.
..

.
.
.
#
.
.
#
.
.
.
..

#
.
.
#
.
.
.
..

.
#
.
.
#
.
.
.
.
#
..

.
.
#
.
.
.
#
.
.
.#

.
..

.
.
#
.
.
#.

.
.
.
.
#
.
.
.
.
#
.
.
#
.
.
..

.
#
.
.
#
.
.
.
.
.
#
.
.
#
.
.
.
.
.
#
.
#
.
.
.
.
.
.
#.

.
.#

.
..

.
.
#
.
#
.
.
.
..

#
.
..

#
.
..

.
#
.
.
#
.
..

..

#
.
.
#
.
.
.
.
#
.
..

#
.
.
..

#
.
.
#
.
..

..

#
.
.
#
.
.
.
.
.
#
..

#
.
.
.
.
.
.
#
.
#
..

..

.
.#

.
#
.
.
.
.
.
